## Задание 1

In [3]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston

boston = load_boston()
data = boston['data']

feature_names = boston['feature_names']
X = pd.DataFrame(data, columns=feature_names)
# X
# boston.keys()
# boston['target']

y = pd.DataFrame(boston['target'], columns=['price'])
# y

c:\users\александр\desktop\npdvirt\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this case special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California ho

In [84]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
# y_test

In [5]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
# type(lr)
lr.fit(X_train, y_train)

LinearRegression()

In [6]:
y_predict = lr.predict(X_test)
# y_predict
system_check = pd.DataFrame({
    'y_test' : y_test['price'],
    'y_pred' : y_predict.flatten()
})

In [7]:
from sklearn.metrics import r2_score

r2 = r2_score(system_check['y_test'], system_check['y_pred'])
r2

0.7112260057484912

## Задание 2

In [8]:
import warnings

warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators = 1000, max_depth = 12, random_state = 42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
model.fit(X_train, y_train.values[:,0])

y_predict = model.predict(X_test)

# system_check = pd.DataFrame({
#     'y_test' : y_test['price'],
#     'y_pred' : y_predict
# })

model.score(X_test, y_test)

0.87472606157312

##### Таким образом, случайный лес демонстрирует более высокий R2, чем линейная модель, то есть объясняет большую долю дисперсии

## Задание 3

In [9]:
?RandomForestRegressor

In [10]:
predictor_values = model.feature_importances_

sum_predictors = np.sum(predictor_values)

##### сумма всех предикторов:

In [11]:
sum_predictors

1.0

##### самые важные показатели - LSTAT и RM

In [12]:
predictors = pd.DataFrame(data = {'values' : predictor_values}, index = model.feature_names_in_)
predictors.sort_values(by='values', ascending = False).head(2)

,values
LSTAT,0.415847
RM,0.402682


## Задание 4

In [13]:
a = pd.read_csv('../dataset/creditcard.csv')


In [14]:
from sklearn.model_selection import GridSearchCV

In [15]:
a.value_counts('Class', normalize = True)

Class
0    0.998273
1    0.001727
dtype: float64

###### Выборка явно несбалансированная, мошеннических операций гораздо меньше, чем честных

In [16]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [17]:
np.sum(a.value_counts())

284807

##### Количество non-null данных равно количеству всех столбцов. Судя по всему, пропусков нет

In [85]:
pd.options.display.max_columns = 100
a.head(10)
X = a.loc[:, a.columns != 'Class']
y = pd.Series(a['Class'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 100, stratify = y)
# type(X_train)
# type(X_test)
# type(y_train)
# type(y_test)

### X_train, X_test - это dataframe, y_train, y_test - это series

In [35]:
from sklearn.ensemble import RandomForestClassifier
parameters = [{
    'n_estimators' : [10, 15],
    'max_features' : np.arange(3,5),
    'max_depth' : np.arange(4,7)
}]
classifier = GridSearchCV(
    estimator = RandomForestClassifier(random_state = 100),
    param_grid = parameters,
    scoring = 'roc_auc',
    cv = 3
)

In [36]:
classifier.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=100),
             param_grid=[{'max_depth': array([4, 5, 6]),
                          'max_features': array([3, 4]),
                          'n_estimators': [10, 15]}],
             scoring='roc_auc')

In [37]:
classifier.best_params_

{'max_depth': 6, 'max_features': 3, 'n_estimators': 15}

In [102]:
y_predict = classifier.predict_proba(X_test)
# y_predict_proba = y_predict
y_predict_proba = y_predict[:,1]

In [103]:
from sklearn.metrics import roc_auc_score
# y_test.value_counts('Class')
roc_auc_score(y_test, y_predict_proba)

0.9462664156037156

## Допзадание

#### 1. Загрузите датасет Wine из встроенных датасетов sklearn.datasets с помощью функции load_wine в переменную data

In [104]:
from sklearn.datasets import load_wine

#### 2. Полученный датасет не является датафреймом. Это структура данных, имеющая ключи аналогично словарю. Просмотрите тип данных этой структуры данных и создайте список data_keys, содержащий ее ключи.

In [112]:
wine = load_wine()
type(wine)

sklearn.utils.Bunch

In [113]:
wine.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names'])

In [114]:
data_keys = wine.keys()

#### 3. Просмотрите данные, описание и названия признаков в датасете. Описание нужно вывести в виде привычного, аккуратно оформленного текста, без обозначений переноса строки, но с самими переносами и т.д.

In [115]:
wine['data']

array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
        1.065e+03],
       [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
        1.050e+03],
       [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
        1.185e+03],
       ...,
       [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
        8.350e+02],
       [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
        8.400e+02],
       [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
        5.600e+02]])

In [117]:
print(wine['DESCR'])

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

In [118]:
wine['feature_names']

['alcohol',
 'malic_acid',
 'ash',
 'alcalinity_of_ash',
 'magnesium',
 'total_phenols',
 'flavanoids',
 'nonflavanoid_phenols',
 'proanthocyanins',
 'color_intensity',
 'hue',
 'od280/od315_of_diluted_wines',
 'proline']

##### 4. Сколько классов содержит целевая переменная датасета? Выведите названия классов.


In [136]:
pd.value_counts(wine['target'])

1    71
0    59
2    48
dtype: int64

In [133]:
wine['target_names']

array(['class_0', 'class_1', 'class_2'], dtype='<U7')

##### 5. На основе данных датасета (они содержатся в двумерном массиве Numpy) и названий признаков создайте датафрейм под названием X.

In [194]:
X = pd.DataFrame(wine['data'], columns = wine['feature_names'])
X

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0


#### 6. Выясните размер датафрейма X и установите, имеются ли в нем пропущенные значения.

In [143]:
np.size(X)

2314

In [148]:
# X.isnull()
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       178 non-null    float64
 1   malic_acid                    178 non-null    float64
 2   ash                           178 non-null    float64
 3   alcalinity_of_ash             178 non-null    float64
 4   magnesium                     178 non-null    float64
 5   total_phenols                 178 non-null    float64
 6   flavanoids                    178 non-null    float64
 7   nonflavanoid_phenols          178 non-null    float64
 8   proanthocyanins               178 non-null    float64
 9   color_intensity               178 non-null    float64
 10  hue                           178 non-null    float64
 11  od280/od315_of_diluted_wines  178 non-null    float64
 12  proline                       178 non-null    float64
dtypes: fl

##### Судя по всему, нулевых значений тут нет

#### 7. Добавьте в датафрейм поле с классами вин в виде чисел, имеющих тип данных numpy.int64. Название поля - 'target'.

In [189]:
target = pd.DataFrame({'target' : wine['target']}, dtype = 'int64')
target.fillna(0)

,target
0,0
1,0
2,0
3,0
4,0
...,...
173,2
174,2
175,2
176,2
